# Implementation of and Exploring Bounds for Zaremba Index Calculation 

First we must import

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

# Import Calculations for Zaremba Index
from math_help import rho_box1_numpy, rho_box_bruteforce, rho_box_lyness

# Import Dataframe Helper
from experiment import bound_sufficiency_df

## Zaremba Index Calculations
- Lyness: The universal truth, Algorithm 2 in "A Search Program for Finding Optimal Integration Lattices*"
- Bruteforce: Bruteforce Calculation that goes through each value -M M to compute
- Numpy: More efficient way to calculate index, with bound -M and M still

Now we will quickly show that Bruteforce, Numpy, and Lyness give the same values

In [5]:
# Test Cases (N, alpha)
golden_ratio = (math.sqrt(5) - 1) / 2

CASES = [
    (5, golden_ratio),      # 1. Small N + Golden Ratio
    (89, golden_ratio),     # 2. Classic Fibonacci + Golden Ratio
    (100, 0.123456789)      # 3. Random Alpha 
]
methods = ["numpy", "bruteforce", "lyness"]
rows = []

# Quick Test Cases to Show that the Bruteforce, Numpy, Lyness all give Same Result searching through N
for N, alpha in CASES:
    M = N
    for method in methods:
        if method == "numpy":
            rho, k_star, h_star = rho_box1_numpy(N, alpha, M)
        elif method == "bruteforce":
            rho, k_star, h_star = rho_box_bruteforce(N, alpha, M)
        elif method == "lyness":
            rho, k_star, h_star = rho_box_lyness(N, alpha)
        
        rows.append({
            "Method": method,
            "N": N,
            "alpha": alpha,
            "M": M,
            "rho": rho,
            "k1_star": k_star[0],
            "k2_star": k_star[1],
            "h1_star": h_star[0],
            "h2_star": h_star[1],
        })
    
df = pd.DataFrame(rows)
df

,Method,N,alpha,M,rho,k1_star,k2_star,h1_star,h2_star
0,numpy,5,0.618034,5,1.909830,-1,-1,-1.909830,-1.0
1,bruteforce,5,0.618034,5,1.909830,-1,-1,-1.909830,-1.0
2,lyness,5,0.618034,5,1.909830,1,1,1.909830,1.0
3,numpy,89,0.618034,89,33.994975,-1,-1,-33.994975,-1.0
4,bruteforce,89,0.618034,89,33.994975,-1,-1,-33.994975,-1.0
5,lyness,89,0.618034,89,33.994975,1,1,33.994975,1.0
6,numpy,100,0.123457,100,9.876550,-8,-1,-1.234569,-8.0
7,bruteforce,100,0.123457,100,9.876550,-8,-1,-1.234569,-8.0
8,lyness,100,0.123457,100,9.876550,8,1,1.234569,8.0


## Results 
From our result we can observe that all 3 methods gave the same answer for our test cases
Thus, all three methods are valid

**Note**: Lyness always gives the positive equivalent of the others as it searches through 0, M+1 due to symmetry

### Next Step: Investigate Bounds for M 
We will look at how our output change for different multiples of N: (N/4, N/2, N, 2N, 4N, 8N)

In [31]:
# Establish Alphas, Ns, Multipliers
alphas = {
    "golden": (math.sqrt(5) - 1) / 2,
    "sqrt2": math.sqrt(2) - 1,
    "e": math.e - 2,
    "near-rational1": 1/3 + 0.0001,
    "near-rational2": 0.500001
}

Ns = [10,50, 100, 200, 400]
M_multipliers = [0.25, 0.5, 1, 2, 4]

In [37]:
all_results = []
benchmark_results = []
# Show all results in a dataframe
for alpha_name, alpha in alphas.items():
    for N in Ns:
        rho_lyness, k_lyness, _ = rho_box_lyness(N, alpha)

        benchmark_results.append({
            "alpha_name": alpha_name,
            "N": N,
            "rho_ground_truth": rho_lyness,
            "k_true": k_lyness
        })
        for mult in M_multipliers:
            M = max(1, int(round(mult * N)))  # ensure M >= 1
            rho, k_star, h_star = rho_box1_numpy(
                N=N,
                alpha=alpha,
                M=M,
            )

            all_results.append({
                "alpha_name": alpha_name,
                "alpha": alpha,
                "N": N,
                "M_multiplier": mult,
                "M": M,
                "rho": rho,
                "k1_star": k_star[0],
                "k2_star": k_star[1],
            })

df_all = pd.DataFrame(all_results)
df_benchmarks = pd.DataFrame(benchmark_results)

df_golden = (
    df_all[df_all["alpha_name"] == "golden"]
    .sort_values(["N", "M_multiplier"])
    .reset_index(drop=True)
)

df_sqrt2 = (
    df_all[df_all["alpha_name"] == "sqrt2"]
    .sort_values(["N", "M_multiplier"])
    .reset_index(drop=True)
)

df_e = (
    df_all[df_all["alpha_name"] == "e"]
    .sort_values(["N", "M_multiplier"])
    .reset_index(drop=True)
)

df_nearrational1 = (
    df_all[df_all["alpha_name"] == "near-rational1"]
    .sort_values(["N", "M_multiplier"])
    .reset_index(drop=True)
)
df_nearrational2 = (
    df_all[df_all["alpha_name"] == "near-rational2"]
    .sort_values(["N", "M_multiplier"])
    .reset_index(drop=True)
)

In [38]:
# Benchmark Results with Lyness Calculation, compare to rest of results
df_benchmarks = pd.DataFrame(benchmark_results)
df_benchmarks

,alpha_name,N,rho_ground_truth,k_true
0,golden,10,3.819660,"(1, 1)"
1,golden,50,19.098301,"(1, 1)"
2,golden,100,38.196601,"(1, 1)"
3,golden,200,76.393202,"(1, 1)"
4,golden,400,152.786405,"(1, 1)"
5,sqrt2,10,3.431458,"(2, 1)"
6,sqrt2,50,17.157288,"(2, 1)"
7,sqrt2,100,34.314575,"(2, 1)"
8,sqrt2,200,68.629150,"(2, 1)"
9,sqrt2,400,137.258300,"(2, 1)"


In [33]:
# Golden Ration Ouput
df_golden

,alpha_name,alpha,N,M_multiplier,M,rho,k1_star,k2_star
0,golden,0.618034,10,0.25,2,3.819660,-1,-1
1,golden,0.618034,10,0.50,5,3.819660,-1,-1
2,golden,0.618034,10,1.00,10,3.819660,-1,-1
3,golden,0.618034,10,2.00,20,3.819660,-1,-1
4,golden,0.618034,10,4.00,40,3.819660,-1,-1
5,golden,0.618034,50,0.25,12,19.098301,-1,-1
6,golden,0.618034,50,0.50,25,19.098301,-1,-1
7,golden,0.618034,50,1.00,50,19.098301,-1,-1
8,golden,0.618034,50,2.00,100,19.098301,-1,-1
9,golden,0.618034,50,4.00,200,19.098301,-1,-1


In [34]:
## Sqrt2 Output
df_sqrt2


,alpha_name,alpha,N,M_multiplier,M,rho,k1_star,k2_star
0,sqrt2,0.414214,10,0.25,2,3.431458,-2,-1
1,sqrt2,0.414214,10,0.50,5,3.431458,-2,-1
2,sqrt2,0.414214,10,1.00,10,3.431458,-2,-1
3,sqrt2,0.414214,10,2.00,20,3.431458,-2,-1
4,sqrt2,0.414214,10,4.00,40,3.431458,-2,-1
5,sqrt2,0.414214,50,0.25,12,17.157288,-2,-1
6,sqrt2,0.414214,50,0.50,25,17.157288,-2,-1
7,sqrt2,0.414214,50,1.00,50,17.157288,-2,-1
8,sqrt2,0.414214,50,2.00,100,17.157288,-2,-1
9,sqrt2,0.414214,50,4.00,200,17.157288,-2,-1


In [35]:
## e Output
df_e

,alpha_name,alpha,N,M_multiplier,M,rho,k1_star,k2_star
0,e,0.718282,10,0.25,2,2.817182,-1,-1
1,e,0.718282,10,0.50,5,2.817182,-1,-1
2,e,0.718282,10,1.00,10,2.817182,-1,-1
3,e,0.718282,10,2.00,20,2.817182,-1,-1
4,e,0.718282,10,4.00,40,2.817182,-1,-1
5,e,0.718282,50,0.25,12,9.790480,-7,-5
6,e,0.718282,50,0.50,25,9.790480,-7,-5
7,e,0.718282,50,1.00,50,9.790480,-7,-5
8,e,0.718282,50,2.00,100,9.790480,-7,-5
9,e,0.718282,50,4.00,200,9.790480,-7,-5


In [36]:
## Near Rational #1 Output
df_nearrational1

,alpha_name,alpha,N,M_multiplier,M,rho,k1_star,k2_star
0,near-rational1,0.333433,10,0.25,2,3.334333,-1,0
1,near-rational1,0.333433,10,0.50,5,3.000000,-3,-1
2,near-rational1,0.333433,10,1.00,10,3.000000,-3,-1
3,near-rational1,0.333433,10,2.00,20,3.000000,-3,-1
4,near-rational1,0.333433,10,4.00,40,3.000000,-3,-1
5,near-rational1,0.333433,50,0.25,12,3.000000,-3,-1
6,near-rational1,0.333433,50,0.50,25,3.000000,-3,-1
7,near-rational1,0.333433,50,1.00,50,3.000000,-3,-1
8,near-rational1,0.333433,50,2.00,100,3.000000,-3,-1
9,near-rational1,0.333433,50,4.00,200,3.000000,-3,-1


In [30]:
## Near Rational #2 Output
df_nearrational2

,alpha_name,alpha,N,M_multiplier,M,rho,k1_star,k2_star
0,near-rational2,0.500001,50,0.25,12,2.0,-2,-1
1,near-rational2,0.500001,50,0.50,25,2.0,-2,-1
2,near-rational2,0.500001,50,1.00,50,2.0,-2,-1
3,near-rational2,0.500001,50,2.00,100,2.0,-2,-1
4,near-rational2,0.500001,50,4.00,200,2.0,-2,-1
5,near-rational2,0.500001,100,0.25,25,2.0,-2,-1
6,near-rational2,0.500001,100,0.50,50,2.0,-2,-1
7,near-rational2,0.500001,100,1.00,100,2.0,-2,-1
8,near-rational2,0.500001,100,2.00,200,2.0,-2,-1
9,near-rational2,0.500001,100,4.00,400,2.0,-2,-1


## Results
From our results it shows that for these specific N and alpha values the critical dual lattice vector
is located very close to the origin. In fact N/4 worked for every single one other than the near-rational2
which is expected as near-rationals tend to have their worst case farther from the origin.

### Summary:
The minimal search bound M required to recover the Zaremba index depends on the size of the minimizing 
integer vector k\*, which in turn is governed by the quality of the best rational approximation of alpha. 
For badly approzimable alphas k\* is extremley small, so M can be much smaller than N without changing result